# Schwarzschild Black Hole Ray Tracing

このノートブックでは、`black_hole_raytracer` ライブラリを使用してシュワルツシルトブラックホールのレイトレーシング画像をインタラクティブに生成します。

In [2]:
println!("Hello world!");

Hello world!


## 1. セットアップ

必要なクレートをインポートします。

In [ ]:
:dep black_hole_raytracer = { path = ".." }
:dep image = "0.25"
:dep tokio = { version = "1", features = ["full"] }

In [ ]:
use black_hole_raytracer::*;
use std::path::Path;

## 2. レンダラーの初期化

GPU コンテキストと BlackHoleRenderer を初期化します。

In [ ]:
// 画像サイズ
let width = 800u32;
let height = 600u32;

// GPU コンテキストの初期化（非同期）
let context = block_on(GpuContext::new()).expect("Failed to create GPU context");

// レンダラーの作成
let mut renderer = BlackHoleRenderer::new_with_context(context, width, height)
    .expect("Failed to create renderer");

println!("レンダラーの初期化が完了しました ({}x{})", width, height);

## 3. カメラとシーンパラメータの設定

レイトレーシングのパラメータを設定します。

In [ ]:
// カメラの位置と向き
let camera_pos = [15.0, 5.0, 0.0];  // ブラックホールから 15 単位離れた位置
let look_at = [0.0, 0.0, 0.0];      // ブラックホールの中心を注視
let up = [0.0, 1.0, 0.0];           // 上方向

let camera = Camera::new(camera_pos, look_at, up);

// シーンパラメータ
let scene = SceneParams {
    black_hole_position: [0.0, 0.0, 0.0],
    schwarzschild_radius: 2.0,  // シュワルツシルト半径
    screen_width: width,
    screen_height: height,
    fov: std::f32::consts::PI / 3.0,  // 視野角 60度
    max_steps: 500,  // レイトレーシングの最大ステップ数
};

println!("カメラ位置: {:?}", camera_pos);
println!("シュワルツシルト半径: {}", scene.schwarzschild_radius);

## 4. レンダリングの実行

レイトレーシングを実行して画像を生成します。

In [ ]:
// レンダリング実行
renderer.render_frame(&camera, &scene);

// 画像データを取得（非同期）
let image_data = block_on(renderer.get_image_data())
    .expect("Failed to get image data");

println!("レンダリングが完了しました ({} bytes)", image_data.len());

## 5. 画像の保存

レンダリング結果を PNG ファイルとして保存します。

In [ ]:
let output_path = "black_hole_render.png";

block_on(renderer.save_image(output_path))
    .expect("Failed to save image");

println!("画像を保存しました: {}", output_path);

## 6. パラメータ調整の例

異なるパラメータで複数の画像を生成してみましょう。

In [ ]:
// 異なる角度から撮影
let angles = [0.0, std::f32::consts::PI / 4.0, std::f32::consts::PI / 2.0];
let radius = 15.0;
let height = 5.0;

for (i, angle) in angles.iter().enumerate() {
    // カメラ位置を計算
    let camera_pos = [
        radius * angle.cos(),
        height,
        radius * angle.sin(),
    ];
    
    let camera = Camera::new(camera_pos, [0.0, 0.0, 0.0], [0.0, 1.0, 0.0]);
    
    // レンダリング
    renderer.render_frame(&camera, &scene);
    
    // 保存
    let output_path = format!("black_hole_angle_{}.png", i);
    block_on(renderer.save_image(&output_path))
        .expect("Failed to save image");
    
    println!("保存完了: {} (角度: {:.2} rad)", output_path, angle);
}

## 7. シュワルツシルト半径の変更

異なるシュワルツシルト半径でレンダリングしてみます。

In [ ]:
// 異なるシュワルツシルト半径
let radii = [1.0, 2.0, 3.0];
let camera = Camera::new([15.0, 5.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.0, 0.0]);

for (i, &schwarzschild_radius) in radii.iter().enumerate() {
    let scene = SceneParams {
        black_hole_position: [0.0, 0.0, 0.0],
        schwarzschild_radius,
        screen_width: width,
        screen_height: height,
        fov: std::f32::consts::PI / 3.0,
        max_steps: 500,
    };
    
    // レンダリング
    renderer.render_frame(&camera, &scene);
    
    // 保存
    let output_path = format!("black_hole_radius_{}.png", i);
    block_on(renderer.save_image(&output_path))
        .expect("Failed to save image");
    
    println!("保存完了: {} (半径: {})", output_path, schwarzschild_radius);
}